# Understanding Word2vec Embedding in Practice
Reference to the [article](https://towardsdatascience.com/understanding-word2vec-embedding-in-practice-3e9b8985953)

* We use Gensim to train word2vec embedding.
* We use NLTK and spaCy to pre-process the text.
* We use t-SNE to visualize high-dimensional data.

In [4]:
import re, string 
import pandas as pd   
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from gensim.models import Word2Vec
%matplotlib inline

df = pd.read_csv('bbc-text.csv')

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    # Remove a sentence if it is only one word long
    if len(text) > 2:
        return ' '.join(word for word in text.split() if word not in STOPWORDS)

df_clean = pd.DataFrame(df.text.apply(lambda x: clean_text(x)))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Yuan Tao/nltk_data'
    - 'c:\\Users\\Yuan Tao\\anaconda3\\envs\\tunglinenv\\nltk_data'
    - 'c:\\Users\\Yuan Tao\\anaconda3\\envs\\tunglinenv\\share\\nltk_data'
    - 'c:\\Users\\Yuan Tao\\anaconda3\\envs\\tunglinenv\\lib\\nltk_data'
    - 'C:\\Users\\Yuan Tao\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
